<a href="https://colab.research.google.com/github/skyadav1989/python-collab-notebooks/blob/main/Python_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Decorators

Decorators are a powerful way to modify or enhance the behavior of functions or methods without permanently altering their code. They are essentially functions that take another function as an argument and return a new function, typically an enhanced version of the original. Decorators are often used for logging, timing, authentication, and more.

In [ ]:
import time

def timer(func):
    """A decorator that prints the time a function takes to execute."""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' took {end_time - start_time:.4f} seconds to execute.")
        return result
    return wrapper

@timer
def long_running_function(n):
    """A sample function that takes some time to run."""
    sum_val = 0
    for i in range(n):
        sum_val += i
    return sum_val

print(f"Result: {long_running_function(10000000)}")

## 2. Generators

Generators are functions that return an iterator. They are especially useful for working with large datasets or infinite sequences because they generate items one by one, on-the-fly, instead of creating all of them in memory at once. This makes them memory-efficient. A function becomes a generator when it uses the `yield` keyword instead of `return`.

In [ ]:
def fibonacci_generator(n):
    """A generator function for the Fibonacci sequence up to n terms."""
    a, b = 0, 1
    count = 0
    while count < n:
        yield a
        a, b = b, a + b
        count += 1

# Using the generator
print("Fibonacci sequence:")
for num in fibonacci_generator(10):
    print(num, end=" ")
print("\n")

# Example of memory efficiency (not creating a full list)
my_generator = fibonacci_generator(1000000) # This doesn't consume memory for all 1M numbers yet
# print(next(my_generator)) # Generates the first number
# print(next(my_generator)) # Generates the second number

## 3. Context Managers (`with` statement)

Context managers allow you to allocate and release resources precisely when you want to. The `with` statement ensures that setup (e.g., opening a file) and teardown (e.g., closing a file) operations are handled automatically, even if errors occur within the block. They make your code cleaner and more robust when dealing with resources like files, network connections, or database connections.

You can create your own context managers using classes with `__enter__` and `__exit__` methods, or by using the `@contextlib.contextmanager` decorator.

In [ ]:
import os

class ChangeDirectory:
    """A custom context manager to temporarily change the current working directory."""
    def __init__(self, new_path):
        self.new_path = os.path.abspath(new_path)
        self.old_path = os.getcwd()

    def __enter__(self):
        print(f"Entering context: Changing directory to {self.new_path}")
        os.chdir(self.new_path)
        return self.new_path

    def __exit__(self): # exc_type, exc_val, exc_tb are for exception handling
        print(f"Exiting context: Changing directory back to {self.old_path}")
        os.chdir(self.old_path)

# Create a temporary directory for demonstration
if not os.path.exists('temp_dir'):
    os.makedirs('temp_dir')

print(f"Current directory: {os.getcwd()}")

with ChangeDirectory('temp_dir') as current_dir:
    print(f"Inside context - Current directory: {os.getcwd()}")
    with open('test_file.txt', 'w') as f:
        f.write('This is a test file.')

print(f"Outside context - Current directory: {os.getcwd()}")

# Clean up the temporary directory and file
os.remove('temp_dir/test_file.txt')
os.rmdir('temp_dir')

## 4. Metaclasses

Metaclasses are a more advanced and less commonly used feature in Python. In Python, everything is an object, and classes themselves are objects. A metaclass is the 'class of a class' – it defines how a class behaves and how it is created. Just as a class creates instances, a metaclass creates classes.

The default metaclass in Python is `type`. You can specify a different metaclass for your class using the `metaclass` argument in the class definition. They are typically used for advanced API design, automatic registration of classes, or injecting specific behaviors into a class upon its creation.

**Note**: Metaclasses are powerful but can make code harder to understand. Use them only when simpler alternatives (like decorators or inheritance) are insufficient.

In [ ]:
class MyMeta(type):
    """A simple metaclass that adds an attribute to any class it creates."""
    def __new__(mcs, name, bases, dct):
        # mcs: the metaclass itself (MyMeta)
        # name: the name of the class being created (e.g., 'MyClass')
        # bases: a tuple of base classes
        # dct: a dictionary of attributes and methods for the new class

        # Add a new attribute to the class
        dct['added_by_metaclass'] = 'Hello from MyMeta!'

        # Create the class using the default type.__new__
        return super().__new__(mcs, name, bases, dct)

class MyClass(metaclass=MyMeta):
    """A class that uses MyMeta as its metaclass."""
    def __init__(self, value):
        self.value = value

    def print_value(self):
        print(f"Instance value: {self.value}")

# Create an instance of MyClass
obj = MyClass(10)

# Access the attribute added by the metaclass
print(f"Attribute from metaclass: {obj.added_by_metaclass}")

# Verify that the attribute is also on the class itself
print(f"Class attribute from metaclass: {MyClass.added_by_metaclass}")

obj.print_value()

## 5. Closures

A closure is a function object that remembers values in its enclosing scope even if they are no longer in memory. In simpler terms, a closure allows a function to access and manipulate variables from an outer (enclosing) function, even after the outer function has finished execution and its local scope has been destroyed. This is achieved by creating a nested function that 'closes over' the variables of its parent function.

Closures are commonly used for data hiding, implementing decorators (as seen earlier), and creating function factories.

In [1]:
def outer_function(message):
    """The outer function that defines a message."""
    def inner_function():
        """The inner function (closure) that remembers the message."""
        print(message)
    return inner_function

# Create two closures, each remembering a different message
hello_func = outer_function("Hello from a closure!")
bye_func = outer_function("Goodbye from another closure!")

# Call the closures
hello_func()
bye_func()

# The message variable from outer_function is 'closed over' by inner_function

Hello from a closure!
Goodbye from another closure!


## 6. Partial Function Application (`functools.partial`)

`functools.partial` allows you to 'freeze' some portion of a function's arguments and/or keywords, resulting in a new function with a simplified signature. This is particularly useful when you need to use a function that requires many arguments, but in a specific context, some of those arguments will always be the same. It can make your code more concise and easier to read by creating specialized versions of general functions.

In [ ]:
from functools import partial

def multiply(x, y):
    return x * y

def power(base, exponent):
    return base ** exponent

# Create a new function that always multiplies by 2
double = partial(multiply, 2)
print(f"Double 10: {double(10)}")  # Effectively calls multiply(2, 10)
print(f"Double 5: {double(5)}")    # Effectively calls multiply(2, 5)

# Create a new function that always raises to the power of 3
cube_of = partial(power, exponent=3)
print(f"Cube of 4: {cube_of(4)}") # Effectively calls power(4, exponent=3)
print(f"Cube of 2: {cube_of(2)}") # Effectively calls power(2, exponent=3)

# Example with a more complex function
def greeting(salutation, name, punctuation='.'):
    return f"{salutation}, {name}{punctuation}"

# Create a personalized greeting function
hello_john = partial(greeting, 'Hello', 'John', punctuation='!')
print(hello_john())

# Create a more general friendly greeting
friendly_greeting = partial(greeting, 'Hi', punctuation=' :)')
print(friendly_greeting('Alice'))
print(friendly_greeting('Bob'))

## 7. Descriptors

Descriptors are objects that implement at least one of the descriptor protocol methods (`__get__`, `__set__`, or `__delete__`). They allow you to customize attribute access (getting, setting, or deleting a value) for managed attributes in a class. Descriptors are the mechanism behind many Python features, including methods, `property()`, `classmethod()`, and `staticmethod()`.

They provide a powerful way to add reusable property logic (like validation, type checking, or lazy loading) without writing the same boilerplate code in every getter/setter.

In [ ]:
class PositiveNumber:
    """A descriptor that ensures an attribute is a positive number."""
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        return getattr(obj, self.private_name, None)

    def __set__(self, obj, value):
        if not isinstance(value, (int, float)) or value <= 0:
            raise ValueError(f"{self.public_name} must be a positive number")
        setattr(obj, self.private_name, value)

class Product:
    name = PositiveNumber() # Using the descriptor
    price = PositiveNumber()

    def __init__(self, name, price):
        self.name = name # This will use the descriptor's __set__
        self.price = price # This will use the descriptor's __set__

    def display(self):
        print(f"Product: {self.name}, Price: ${self.price:.2f}")

# Create product instances
p1 = Product("Laptop", 1200.50)
p1.display()

p2 = Product("Mouse", 25)
p2.display()

# Try to set an invalid value (will raise ValueError)
try:
    p1.price = -100
except ValueError as e:
    print(f"Error setting price: {e}")

try:
    p2.name = 0
except ValueError as e:
    print(f"Error setting name: {e}")

# Accessing attributes uses the descriptor's __get__
print(f"Product 1's price: {p1.price}")

## 8. Arbitrary Arguments (`*args` and `**kwargs`)

In Python, you can define functions that accept a variable number of arguments using `*args` and `**kwargs`. This is particularly useful when you don't know beforehand how many arguments will be passed to your function.

*   **`*args` (Non-keyword Arguments)**: This allows a function to accept an arbitrary number of non-keyword arguments. The arguments are gathered into a tuple.
*   **`**kwargs` (Keyword Arguments)**: This allows a function to accept an arbitrary number of keyword (named) arguments. The arguments are gathered into a dictionary.

In [ ]:
def my_function(*args, **kwargs):
    """Demonstrates *args and **kwargs."""
    print("\n--- Inside my_function ---")
    print("Non-keyword arguments (*args):")
    for arg in args:
        print(f"  - {arg}")

    print("Keyword arguments (**kwargs):")
    for key, value in kwargs.items():
        print(f"  - {key}: {value}")
    print("------------------------")

# Example usage
my_function(1, 2, 'hello', True)

my_function(name='Alice', age=30, city='New York')

my_function(10, 'world', value=100, status='success')

# You can also unpack iterables and dictionaries
numbers = [5, 6]
info = {'country': 'USA', 'zip': '10001'}
my_function(*numbers, **info, message='combined call')

## 9. Specialized Collection Types (e.g., `collections.defaultdict`)

Python's built-in `collections` module provides specialized container datatypes that offer alternatives to general-purpose `dict`, `list`, `set`, and `tuple`. These can often improve the functionality and readability of your code.

One very useful type is `defaultdict`:

*   **`collections.defaultdict`**: This is a subclass of `dict` that calls a factory function to supply missing values. When you try to access a key that doesn't exist, instead of raising a `KeyError`, it automatically inserts a value returned by the factory function (e.g., `list`, `int`, `str`, or a custom function) and returns it.

In [ ]:
from collections import defaultdict

# Example 1: Grouping items
data = [('fruit', 'apple'), ('vegetable', 'carrot'), ('fruit', 'banana'), ('dairy', 'milk'), ('vegetable', 'broccoli')]

# Using a regular dictionary would require checking if the key exists
grouped_data_normal_dict = {}
for category, item in data:
    if category not in grouped_data_normal_dict:
        grouped_data_normal_dict[category] = []
    grouped_data_normal_dict[category].append(item)
print(f"Grouped with normal dict: {grouped_data_normal_dict}")

# Using defaultdict (much cleaner)
grouped_data_defaultdict = defaultdict(list)
for category, item in data:
    grouped_data_defaultdict[category].append(item)
print(f"Grouped with defaultdict: {grouped_data_defaultdict}")

print("\n")

# Example 2: Counting occurrences
word_list = ['apple', 'banana', 'apple', 'orange', 'banana', 'apple']

# Using a regular dictionary
word_counts_normal_dict = {}
for word in word_list:
    word_counts_normal_dict[word] = word_counts_normal_dict.get(word, 0) + 1
print(f"Counts with normal dict: {word_counts_normal_dict}")

# Using defaultdict (with int as factory)
word_counts_defaultdict = defaultdict(int)
for word in word_list:
    word_counts_defaultdict[word] += 1
print(f"Counts with defaultdict: {word_counts_defaultdict}")

## 10. Higher-Order Functions

A higher-order function is a function that takes one or more functions as arguments, returns a function as its result, or both. This concept is fundamental to functional programming paradigms in Python.

Common examples of built-in higher-order functions in Python include `map()`, `filter()`, and `sorted()`, which take a function and an iterable as arguments. Decorators (which we covered) are also a form of higher-order functions.

Using higher-order functions can lead to more concise, readable, and reusable code.

In [2]:
# Example 1: Function that takes another function as an argument
def apply_operation(func, numbers):
    """Applies a given function to each number in a list."""
    return [func(num) for num in numbers]

def square(x):
    return x * x

def cube(x):
    return x ** 3

numbers = [1, 2, 3, 4, 5]
print(f"Original numbers: {numbers}")
print(f"Squared numbers: {apply_operation(square, numbers)}")
print(f"Cubed numbers: {apply_operation(cube, numbers)}")

print("\n")

# Example 2: Function that returns another function
def multiplier(factor):
    """Returns a function that multiplies its argument by 'factor'."""
    def multiply_by_factor(number):
        return number * factor
    return multiply_by_factor

double = multiplier(2)
triple = multiplier(3)

print(f"Double 5: {double(5)}")
print(f"Triple 5: {triple(5)}")

print("\n")

# Example 3: Using built-in higher-order functions
words = ["apple", "banana", "cherry", "date"]

# filter() to get words longer than 5 characters
long_words = list(filter(lambda word: len(word) > 5, words))
print(f"Long words: {long_words}")

# map() to get the length of each word
word_lengths = list(map(len, words))
print(f"Word lengths: {word_lengths}")

Original numbers: [1, 2, 3, 4, 5]
Squared numbers: [1, 4, 9, 16, 25]
Cubed numbers: [1, 8, 27, 64, 125]


Double 5: 10
Triple 5: 15


Long words: ['banana', 'cherry']
Word lengths: [5, 6, 6, 4]
